<a href="https://colab.research.google.com/github/speedsis/estudo-IA/blob/master/Hackatran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorithimo desenvolvido para o Hackatran 2020 **
Autor: JOSE MAXWELL TAVARES
22 de Novembro de 2020

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/gun_detection/data/

/gdrive/MyDrive/gun_detection/data


In [ ]:
!mkdir test_labels train_labels

In [ ]:
!ls annotations/* | sort -R | head -3 | xargs -I{} mv {} test_labels/
!ls annotations/* | xargs -I{} mv {} train_labels/

# Moves the first 400/2000 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -2 | xargs -I{} mv {} test_labels/

# Moves the rest of labels '1600' labels to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/

In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

In [ ]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [ ]:
#!pip3 install tensorflow==1.15.0

!pip uninstall -y tensorflow
!pip install tensorflow==1.15.0


In [ ]:
print(tf.__version__)

1.15.0


In [ ]:
%cd /gdrive/MyDrive/gun_detection/

/gdrive/MyDrive/gun_detection


In [ ]:
!wget https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS.zip

--2020-11-23 00:03:37--  https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS.zip
Resolving sci2s.ugr.es (sci2s.ugr.es)... 150.214.190.154
Connecting to sci2s.ugr.es (sci2s.ugr.es)|150.214.190.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250005059 (238M) [application/zip]
Saving to: ‘WeaponS.zip’

WeaponS.zip         100%[===================>] 238.42M  11.0MB/s    in 22s     

2020-11-23 00:04:00 (10.8 MB/s) - ‘WeaponS.zip’ saved [250005059/250005059]



In [ ]:
!unzip -q WeaponS.zip

In [ ]:
!wget https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS_bbox.zip

--2020-11-23 00:04:57--  https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS_bbox.zip
Resolving sci2s.ugr.es (sci2s.ugr.es)... 150.214.190.154
Connecting to sci2s.ugr.es (sci2s.ugr.es)|150.214.190.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1420022 (1.4M) [application/zip]
Saving to: ‘WeaponS_bbox.zip’

WeaponS_bbox.zip    100%[===================>]   1.35M  2.01MB/s    in 0.7s    

2020-11-23 00:04:58 (2.01 MB/s) - ‘WeaponS_bbox.zip’ saved [1420022/1420022]



In [ ]:
!unzip -q WeaponS_bbox.zip

In [ ]:
!mkdir data

In [ ]:
!mkdir data/images data/train_labels data/test_labels

In [ ]:
!mv WeaponS/* data/images

In [ ]:
!mv WeaponS_bbox/* data/train_labels

In [ ]:
!rm -rf WeaponS_bbox.zip  WeaponS.zip WeaponS/  WeaponS_bbox/

In [ ]:
!ls data/train_labels/* | sort -R | head -600 | xargs -I{} mv {} data/test_labels

In [ ]:
ls -1 data/train_labels/ | wc -l

1785


In [ ]:
ls -1 data/test_labels/ | wc -l

600


In [ ]:
%cd /gdrive/MyDrive/gun_detection/data/

/gdrive/MyDrive/gun_detection/data


In [ ]:
images_extension = 'jpg'

In [ ]:
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text + '.' + images_extension,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.
for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: 'Gun'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [ ]:
!cat label_map.pbtxt

item {
    id: 1
    name: 'pistol'
    display_name: 'Gun'
 }

In [ ]:
ls -l

total 138
drwx------ 2 root root  4096 Nov 23 00:15 images/
-rw------- 1 root root    62 Nov 23 00:43 label_map.pbtxt
drwx------ 2 root root  4096 Nov 23 00:18 test_labels/
-rw------- 1 root root 32386 Nov 23 00:43 test_labels.csv
drwx------ 2 root root  4096 Nov 23 00:18 train_labels/
-rw------- 1 root root 95418 Nov 23 00:39 train_labels.csv


In [ ]:
%cd /gdrive/MyDrive/gun_detection/data/

/gdrive/MyDrive/gun_detection/data


In [ ]:
images_path = 'images'

In [ ]:
for CSV_FILE in ['train_labels.csv', 'test_labels.csv']:
  with open(CSV_FILE, 'r') as fid:  
      print('[*] Checking file:', CSV_FILE) 
      file = csv.reader(fid, delimiter=',')
      first = True 
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          if error == True:
              error_cnt += 1
              error = False         
          if first == True:
              first = False
              continue     
          cnt += 1      
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])     
          path = os.path.join(images_path, name)
          img = cv2.imread(path)         
          if type(img) == type(None):
              error = True
              print('Could not read image', img)
              continue     
          org_height, org_width = img.shape[:2]     
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)     
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height) 
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)  
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")

[*] Checking file: train_labels.csv
XMIN > org_width for file armas (2815).jpg
XMAX > org_width for file armas (2815).jpg
YMIN > org_height for file armas (2815).jpg
YMAX > org_height for file armas (2815).jpg
Error for file: armas (2815).jpg


Checked 2066 files and realized 1 errors
-----
[*] Checking file: test_labels.csv

Checked 703 files and realized 0 errors
-----


In [ ]:
#temos apenas uma imagem com a posição incorreta da caixa, poderíamos apenas removê-la
#remover a images

rm images/'armas (2815).jpg''

In [ ]:
#remover a entrada para ele no csv dessa imagem também

#porque fizemos uma divisão aleatória dos dados, não sabemos se acabou sendo um treinamento ou teste
# vamos remover a imagem de ambos.

#Treinamento
#reading the training csv
df = pd.read_csv('/gdrive/MyDrive/gun_detection/data/train_labels.csv')
# removendo armas (2815).jpg
df = df[df['filename'] != 'armas (2815).jpg']
#resetando   index
df.reset_index(drop=True, inplace=True)
#salvando df
df.to_csv('/gdrive/MyDrive/gun_detection/data/train_labels.csv')


#testando
#lendo e testando csv
df = pd.read_csv('/gdrive/MyDrive/gun_detection/data/test_labels.csv')
# removendo armas (2815).jpg
df = df[df['filename'] != 'armas (2815).jpg']
#resetando index
df.reset_index(drop=True, inplace=True)
#salvando  o df
df.to_csv('/gdrive/MyDrive/gun_detection/data/test_labels.csv')

#Só para a memória
df = None

Downloading e Preparat Tensorflow model

In [ ]:
# Downlaods Tenorflow
%cd /gdrive/MyDrive/gun_detection/
!git clone --q https://github.com/tensorflow/models.git

/gdrive/MyDrive/gun_detection
fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
%cd /gdrive/MyDrive/gun_detection/models/research

/gdrive/MyDrive/gun_detection/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
os.environ['PYTHONPATH'] += ':./:./slim/'

In [ ]:
!pip install tf_slim
!pip install tf-models-official

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [ ]:
!python3 object_detection/builders/model_builder_test.py

2020-11-23 01:34:34.915342: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


# **Gerando Tf record**

In [ ]:
from object_detection.utils import dataset_util
%cd /gdrive/MyDrive/gun_detection/models/

DATA_BASE_PATH = '/gdrive/MyDrive/gun_detection/data/'
image_dir = DATA_BASE_PATH +'images/'

def class_text_to_int(row_label):
		if row_label == 'pistol':
				return 1
		else:
				None


def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))

/gdrive/MyDrive/gun_detection/models
Successfully created the TFRecords: /gdrive/MyDrive/gun_detection/data/train_labels.record
Successfully created the TFRecords: /gdrive/MyDrive/gun_detection/data/test_labels.record


In [ ]:
ls -lX /gdrive/MyDrive/gun_detection/data/

total 200595
drwx------ 2 root root      4096 Nov 23 00:15 images/
drwx------ 2 root root      4096 Nov 23 00:18 test_labels/
drwx------ 2 root root      4096 Nov 23 00:18 train_labels/
-rw------- 1 root root     35089 Nov 23 01:07 test_labels.csv
-rw------- 1 root root    104535 Nov 23 01:07 train_labels.csv
-rw------- 1 root root        62 Nov 23 00:43 label_map.pbtxt
-rw------- 1 root root  52174464 Nov 23 01:38 test_labels.record
-rw------- 1 root root 153080396 Nov 23 01:38 train_labels.record


#MOdelo treinamento 

In [ ]:
%cd /gdrive/MyDrive/gun_detection/models/research

/gdrive/MyDrive/gun_detection/models/research


In [ ]:
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
    }
}

In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

In [ ]:
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools

# **Instalar pacotes requeridos**

In [ ]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

from google.colab import files

# Importando Bibliotecas:

In [ ]:
#!pip3 install tensorflow==1.15.0

print(tf.__version__)

2.3.0


# transformar XML PARA CSV

In [ ]:
%cd /gdrive/MyDrive/object_detection/data/


/gdrive/MyDrive/object_detection/data


In [ ]:
images_extension = 'jpg'

In [ ]:
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text + '.' + images_extension,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.
for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: 'Gun'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [ ]:
!cat label_map.pbtxt

In [ ]:
ls -l

total 20
drwx------ 2 root root 4096 Nov 22 19:48 annotations/
drwx------ 2 root root 4096 Nov 22 17:53 images/
-rw------- 1 root root  261 Nov 22 19:50 label_map.pbtxt
drwx------ 2 root root 4096 Nov 22 19:48 test_labels/
-rw------- 1 root root  955 Nov 22 19:50 test_labels.csv
drwx------ 2 root root 4096 Nov 22 19:48 train_labels/
-rw------- 1 root root 2350 Nov 22 19:50 train_labels.csv


In [ ]:
%cd /gdrive/MyDrive/object_detection/

/gdrive/MyDrive/object_detection


In [ ]:
!git clone --q https://github.com/tensorflow/models.git

In [ ]:
%cd /gdrive/MyDrive/object_detection/models/research/

/gdrive/MyDrive/object_detection/models/research


In [ ]:
%cd /gdrive/MyDrive/object_detection/models/research

/gdrive/MyDrive/object_detection/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
os.environ['PYTHONPATH'] += ':./:./slim/'

In [ ]:
!pip install --upgrade tf-slim

In [ ]:
!pip install tf-models-official

In [ ]:
!python3 object_detection/builders/model_builder_test.py

2020-11-22 20:15:08.781278: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
from object_detection.utils import dataset_util

In [ ]:
%cd /gdrive/MyDrive/object_detection/models


/gdrive/MyDrive/object_detection/models


In [ ]:
DATA_BASE_PATH = '/gdrive/MyDrive/object_detection/data/'
image_dir = DATA_BASE_PATH +'images/'

In [ ]:
def class_text_to_int(row_label):
		if row_label == 'pistol':
				return 1
		else:
				None


def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))

NotFoundError: ignored